# Género y tecnología: ¿qué eligen los estudiantes?

In [799]:
from collections import namedtuple

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from scipy import stats
import seaborn as sns
import matplotlib
import ipywidgets as widgets
from ipywidgets import interact
import folium
from folium.plugins import DualMap 
from datetime import datetime
from IPython.display import HTML, display

# módulos propios
from scripts import mapas, pygal_helpers, georef, download
from scripts.pygal_helpers import df_to_pygal_line, get_pygal_html
from scripts.helpers import read_csv, format_pct, get_bins

matplotlib.style.use("ggplot")

## Evolución de género en grupo de carreras

In [800]:
df_evolucion_carreras = read_csv('grupos_evolucion_genero.csv')
df_evolucion_grupos_carreras = read_csv('grupos_carreras_evolucion_genero.csv')

df_evolucion_carreras.head(10)

,anio,carrera_grupo,estudiantes_total,estudiantes_varones,estudiantes_mujeres,nuevos_inscriptos_total,nuevos_inscriptos_varones,nuevos_inscriptos_mujeres,egresados_total,egresados_varones,egresados_mujeres,estudiantes_mujeres_pct,inscripciones_mujeres_pct,estudiantes_hombres_pct
0,2011,Arquitectura y Urbanismo,60050.0,31064.0,28986.0,9645.0,4797.0,4848.0,3063.0,1412.0,1651.0,0.482698,0.502644,0.517302
1,2011,Artes,39177.0,17948.0,21229.0,10031.0,4935.0,5096.0,1251.0,460.0,791.0,0.541874,0.508025,0.458126
2,2011,Comunicacion,111118.0,39992.0,71126.0,24996.0,9730.0,15266.0,6950.0,2087.0,4863.0,0.640094,0.610738,0.359906
3,2011,Diseño,51974.0,18337.0,33637.0,10856.0,3711.0,7145.0,2825.0,820.0,2005.0,0.647189,0.658161,0.352811
4,2011,Economicas,307461.0,144518.0,162943.0,64276.0,31566.0,32710.0,18404.0,8706.0,9698.0,0.529963,0.508899,0.470037
5,2011,Educacion,134797.0,32001.0,102796.0,40811.0,10557.0,30254.0,8380.0,1776.0,6604.0,0.762599,0.741320,0.237401
6,2011,Exactas,23234.0,11244.0,11990.0,5279.0,2429.0,2850.0,1454.0,622.0,832.0,0.516054,0.539875,0.483946
7,2011,"Hoteleria, Gastronomia y Turismo",25263.0,8040.0,17223.0,6733.0,2333.0,4400.0,1467.0,348.0,1119.0,0.681748,0.653498,0.318252
8,2011,Informatica,160535.0,129875.0,30660.0,36354.0,30258.0,6096.0,7329.0,5539.0,1790.0,0.190986,0.167684,0.809014
9,2011,Ingenieria,121803.0,99148.0,22655.0,25275.0,20342.0,4933.0,5707.0,4526.0,1181.0,0.185997,0.195173,0.814003


In [801]:
# NOTAS: Comparacion de valores absolutos de hombres y mujeres - nuevas inscripciones/ filtro por carrera (eje primaro y secundario??)

In [802]:
df_evolucion_carreras = df_evolucion_carreras[df_evolucion_carreras.anio != 2010]

In [803]:
df_evolucion_grupos_carreras = df_evolucion_grupos_carreras[df_evolucion_grupos_carreras.anio !=2010]

### ¿Cómo evolucionó la proporción de estudiantes femeninos en distintas disciplinas?

In [804]:
# gráfico de evolución hombres / mujeres por grupo de carreras

# widget: seleccionar grupo temático de carreras
variables_carreras_evolucion = tuple(df_evolucion_carreras.carrera_grupo.unique())
widget_grupos=widgets.SelectMultiple(
    options=sorted(variables_carreras_evolucion),
    value=["Informatica"],
    description='Areas Tematicas',
    disabled=False
)

# widget: elegir indicador
variables_indicadores_line = {
    "Ratio mujeres": "estudiantes_mujeres_pct",
    "Ratio hombres": "estudiantes_hombres_pct"
}
widget_indicador = widgets.ToggleButtons(
    options=variables_indicadores_line.items(),
    description="Indicador : ",
    value="estudiantes_mujeres_pct"
)

@interact(carrera_grupo=widget_grupos, indicador=widget_indicador)
def generate_chart_evolucion_grupos(carrera_grupo, indicador):
    
    serie_grupo_carrera = df_evolucion_carreras[
        df_evolucion_carreras.carrera_grupo.isin(carrera_grupo)][[
            "anio", 'carrera_grupo',  indicador]]

    serie_grupo_carrera[indicador]=serie_grupo_carrera[indicador].apply(format_pct)
    
    df_pivot_serie_grupo_carrera = serie_grupo_carrera.pivot_table(
        index='anio', columns='carrera_grupo', values=indicador)
    
    return HTML(get_pygal_html(df_to_pygal_line(df_pivot_serie_grupo_carrera)))
        
#NOTAS: aclarar en nota que los valores son porcentajes

interactive(children=(SelectMultiple(description='Areas Tematicas', index=(8,), options=('Arquitectura y Urban…

In [805]:
# gráfico de evolución de las carreras dentro de un grupo

@interact(carrera_grupo=widget_grupos) #widget diferente
def generate_chart_evolucion_titulos(carrera_grupo):
    
    serie_grupo_carrera = df_evolucion_grupos_carreras[
        df_evolucion_grupos_carreras.carrera_grupo.isin(carrera_grupo)][[
        "anio", 'Carrera_normalizada',  "estudiantes_mujeres_pct"]]
    df_pivot_serie_grupo_carrera = serie_grupo_carrera.pivot_table(
        index='anio', columns='Carrera_normalizada', values='estudiantes_mujeres_pct')
    
    return HTML(get_pygal_html(df_to_pygal_line(df_pivot_serie_grupo_carrera)))
        

interactive(children=(SelectMultiple(description='Areas Tematicas', index=(8,), options=('Arquitectura y Urban…

In [806]:
#agregar mas de un filtro para ir comparando
#modificar en ETL abogacia y derecho
#apertura por carreras para entender donde hace el pico y decrecimiento tech - cambiar pKEY por carrera y grupo carrera - grafico exploratorio
#segunda tabla de ETL donde haya grupo carrera (carrera) + universidades
#grafico con valores absolutos - apilados o algo que represente orden de magnitud año por default 

## Comparación de género por provincias y universidades

In [807]:
df_universidades = pd.read_csv('data/output/universidades_mapa.csv')
df_evolucion_ubicacion = pd.read_csv('data/output/ubicacion_evolucion_genero.csv', 
                                     dtype={'provincia_id': str})
df_evolucion_ubicacion['provincia_id'] = df_evolucion_ubicacion.provincia_id.str.zfill(2)

In [808]:
argentina_coords = [-40, -64]
osm_no_labels = "https://tiles.wmflabs.org/osm-no-labels/{z}/{x}/{y}.png"
ign_labels = "http://wms.ign.gob.ar/geoserver/gwc/service/tms/1.0.0/capabaseargenmap@EPSG%3A3857@png/{z}/{x}/{y}.png"

## Indicadores por provincia

In [809]:
INDICADORES_HEATMAP = {
    "estudiantes_mujeres_pct" : "Ratio de estudiantes mujeres",
    "estudiantes_grupo_pct" : "Ratio de estudiantes",
    "estudiantes_varones" : "Total de estudiantes hombres",
    #"estudiantes_mujeres" : "Total de estudiantes mujeres", 
    #"inscriptos_mujeres_pct" : "Ratio de inscriptas mujeres ",
    #"inscriptos_hombres_pct" : "Ratio de inscriptas hombres ",
    #"estudiantes_hombres_pct" : "Ratio de estudiantes hombres ",
}
variables_indicadores = {
    "Ratio mujeres": "estudiantes_mujeres_pct",
    "Ratio grupo": "estudiantes_grupo_pct",
    "Est. hombres": "estudiantes_varones"
}

variables_carreras_heatmap = sorted(df_evolucion_ubicacion.carrera_grupo.unique())
variables_anio_heatmap = sorted(df_evolucion_ubicacion.anio.unique())

widget_indicador = widgets.ToggleButtons(
    options=variables_indicadores.items(),
    description="Indicador : ",
    value="estudiantes_mujeres_pct"
)

widget_carreras = widgets.Dropdown(
    options=variables_carreras_heatmap,
    description="Grupo : ",
    value="Informatica"
)

widget_anio = widgets.Dropdown(
    options=variables_anio_heatmap,
    description="Año : ",
    value=2016
)

widget_scale = widgets.RadioButtons(
    options=['Cuantiles', 'Fija', 'Relativa'],
    description='Escala :',
    value='Relativa'
)

@interact(anio=widget_anio,
          carrera_grupo=widget_carreras,
          indicador=widget_indicador,
          scale_type=widget_scale
         )
def generate_heat_map(anio, carrera_grupo, indicador, scale_type=None):
    mapa_heatmap = folium.Map(location=argentina_coords, zoom_start=4)
    folium.TileLayer(ign_labels, tms=True, attr="IGN").add_to(mapa_heatmap)
    
    df_filter = df_evolucion_ubicacion[(df_evolucion_ubicacion.anio == anio)&
                           (df_evolucion_ubicacion.carrera_grupo == carrera_grupo)]
    
    if scale_type == 'Cuantiles':
        bins = list(df_filter[indicador].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))
    elif scale_type == 'Fija':
        bins = get_bins(df_evolucion_ubicacion[indicador].min(), df_evolucion_ubicacion[indicador].max())
    else:
        bins = 5
       
    indicador_desc = INDICADORES_HEATMAP[indicador]
    folium.Choropleth(
        geo_data="data/input/provincias.geojson", # la capa de provincias en GeoJson
        data=df_filter, # DataFrame con los datos a graficar
        columns=['provincia_id', indicador], # columna con id de geometrías, columna con el dato
        key_on='feature.properties.id', # campo del geojson que tiene el id de las geometrías
        fill_color='YlGn', # escala de colores a usar
        #fill_opacity=0.7, # opacidad del color de relleno
        #line_opacity=0.2, # opacidad de las líneas que separan los polígonos
        legend_name='{} en {} (Año {})'.format(indicador_desc, carrera_grupo, anio), # título de la leyenda de la escala
        bins=bins
    ).add_to(mapa_heatmap)
    #mapas.add_labels_from_column(mapa_heatmap,df_evolucion_ubicacion,"nombre")

    return mapa_heatmap

# NOTAS
#Agregar un tooltip que muestre informacion de la provincia al pasar el cursor. Nombre provincia, estudiantes totales, año, porcentaje de mujeres, cantidad de universidades por provincia.
#Hacer zoom sobre la cuidad de buenos aires. Hacer un grafico nuevo
#min, max y media de la variable analizada por grupo.
#Agregar grafico de evolucion y/y entre provincias por grupo de carreras. Grafico de barras con la composicion de los grupos
#Cambiar los nombres a los indices porque son muy tecnicos. Podriamos usar indice de feminidad, grupo??, masculidad
#Se puede cambiar la escala cuando no hay datos?? (blancos o transparente)
#Tabla con visualizacon y que tenga un orden de ranking para poder identificar facilmente cual es la provincia que mas avanzo

interactive(children=(Dropdown(description='Año : ', index=5, options=(2011, 2012, 2013, 2014, 2015, 2016, 201…

## Indicadores por universidad

In [810]:
def mapear_universidades(mapa, df, lat_col, lon_col, institucion_nombre_col, carreras_grupos_col):
    """ cargar informacion al marker de forma masiva.

        Args:
            lat (float): latitud
            lon (float): longuitud
            institucion_nombre (str): nombre de la universidad
            carreras_grupos (list): lista con las areas de estudios ofrecidas

        """
    
    for row in df_universidades.iterrows():
        mapas.crear_marker(
            mapa,
            row[1]["lat"],
            row[1]["lon"],
            row[1]["institucion_nombre"],
            str(row[1]["carrera_grupo"]).split(",")
        )
        
#NOTAS: agregar feminidad por grupo carrera, elegir si queremos hacer selector por año o elegir 2016.    

In [811]:
mapa_markers = folium.Map(location=argentina_coords, zoom_start=4)
folium.TileLayer(ign_labels, tms=True, attr="IGN").add_to(mapa_markers)

In [812]:
mapear_universidades(
    mapa_markers, df_universidades, 
    "lat","lon","institucion_nombre","carrera_grupo"
)

In [813]:
mapa_markers

# NOTAS
#Mapa + tabla "donde estudio" puedo hacer un filtro por grupo carrera (inverso a lo que estamos haciendo) 
#Cantidad de alumnas mujeres o comparacion de universidades
#agregar al tooltip el indice de feminidad

## Indicadores por universidad Y heatmap

In [814]:
def generate_heat_marker_map(anio, carrera_grupo, indicador, scale_type=None):
    mapa = folium.Map(location=argentina_coords, zoom_start=4)
    folium.TileLayer(ign_labels, tms=True, attr="IGN").add_to(mapa)
    
    df_filter = df_evolucion_ubicacion[(df_evolucion_ubicacion.anio == anio)&
                           (df_evolucion_ubicacion.carrera_grupo == carrera_grupo)]
    
    if scale_type == 'Cuantiles':
        bins = list(df_filter[indicador].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))
    elif scale_type == 'Fija':
        bins = get_bins(df_evolucion_ubicacion[indicador].min(), df_evolucion_ubicacion[indicador].max())
    else:
        bins = 5
       
    indicador_desc = INDICADORES_HEATMAP[indicador]
    folium.Choropleth(
        geo_data="data/input/provincias.geojson", # la capa de provincias en GeoJson
        data=df_filter, # DataFrame con los datos a graficar
        columns=['provincia_id', indicador], # columna con id de geometrías, columna con el dato
        key_on='feature.properties.id', # campo del geojson que tiene el id de las geometrías
        fill_color='YlGn', # escala de colores a usar
        #fill_opacity=0.7, # opacidad del color de relleno
        #line_opacity=0.2, # opacidad de las líneas que separan los polígonos
        legend_name='{} en {} (Año {})'.format(indicador_desc, carrera_grupo, anio), # título de la leyenda de la escala
        bins=bins
    ).add_to(mapa)
    #mapas.add_labels_from_column(mapa_heatmap,df_evolucion_ubicacion,"nombre")
    
    mapear_universidades(
    mapa, df_universidades, 
    "lat","lon","institucion_nombre","carrera_grupo")

    return mapa

In [815]:
generate_heat_marker_map(2016, 'Informatica', 'estudiantes_mujeres_pct')